# Create label for training in future

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

meta = pd.read_csv('./data/Meta/meta_info.csv')
meta.head(5)
# NI= Nodule Image, MA = Mask Original , CN = Clean Nodule , CM = Clean Mask

,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean
0,1,0,0,0001_NI000_slice000,0001_MA000_slice000,5,True,False
1,1,0,1,0001_NI000_slice001,0001_MA000_slice001,5,True,False
2,1,0,2,0001_NI000_slice002,0001_MA000_slice002,5,True,False
3,1,0,3,0001_NI000_slice003,0001_MA000_slice003,5,True,False
4,1,0,4,0001_NI000_slice004,0001_MA000_slice004,5,True,False


In [24]:
def is_nodule(row):
        return 'NI' in row

In [25]:
meta['is_nodule']= meta['original_image'].apply(lambda row: is_nodule(row))

meta.head(5)

,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean,is_nodule
0,1,0,0,0001_NI000_slice000,0001_MA000_slice000,5,True,False,True
1,1,0,1,0001_NI000_slice001,0001_MA000_slice001,5,True,False,True
2,1,0,2,0001_NI000_slice002,0001_MA000_slice002,5,True,False,True
3,1,0,3,0001_NI000_slice003,0001_MA000_slice003,5,True,False,True
4,1,0,4,0001_NI000_slice004,0001_MA000_slice004,5,True,False,True


In [26]:
# Lets separate Clean meta and meta data
clean_meta = meta[meta['is_nodule']==False]
clean_meta.reset_index(inplace=True)
meta = meta[meta['is_nodule']==True]
meta.reset_index(inplace=True)

In [27]:
clean_meta.head(2)

,index,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean,is_nodule


In [28]:
meta.head(2)

,index,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean,is_nodule
0,0,1,0,0,0001_NI000_slice000,0001_MA000_slice000,5,True,False,True
1,1,1,0,1,0001_NI000_slice001,0001_MA000_slice001,5,True,False,True


In [29]:
def is_train(row,train,val,test):
    if row in train:
        return 'Train'
    elif row in val:
        return 'Validation'
    else:
        return 'Test'

In [33]:
clean_patient_id = list(np.unique(clean_meta['patient_id']))
print("Number of clean patients:"+str(len(clean_patient_id)))
meta_patient_id = list(np.unique(meta['patient_id']))
print("Number of meta patients:"+str(len(meta_patient_id)))

Number of clean patients:0
Number of meta patients:5


In [44]:
def create_label_segmentation(meta):
    patient_id = list(np.unique(meta['patient_id']))
    train_patient, test_patient = train_test_split(patient_id,test_size= 0.15)
    train_patient, val_patient = train_test_split(train_patient,test_size= 0.12)
    print(len(train_patient),len(val_patient),len(test_patient))
    
    meta['data_split']= meta['patient_id'].apply(lambda row : is_train(row,train_patient,val_patient,test_patient))
    
    return meta

In [45]:
# We need to train/test split independently for clean_meta, meta
meta = create_label_segmentation(meta)
clean_meta = create_label_segmentation(clean_meta)

3 1 1


ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [46]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(meta[['patient_id', 'data_split']])

    patient_id  data_split
0            1       Train
1            1       Train
2            1       Train
3            1       Train
4            1       Train
5            1       Train
6            1       Train
7            1       Train
8            2       Train
9            2       Train
10           2       Train
11           2       Train
12           2       Train
13           2       Train
14           2       Train
15           2       Train
16           2       Train
17           2       Train
18           2       Train
19           2       Train
20           2       Train
21           2       Train
22           2       Train
23           2       Train
24           2       Train
25           2       Train
26           2       Train
27           2       Train
28           2       Train
29           2       Train
30           2       Train
31           2       Train
32           2       Train
33           2       Train
34           2       Train
35           2       Train
3

In [16]:
# Clean Meta only stores meta information of patients without nodules.
meta.to_csv('/home/LUNG_DATA/meta_csv/meta.csv')
clean_meta.to_csv('/home/LUNG_DATA/meta_csv/clean_meta.csv')